In [1]:
import pandas as pd
#%matplotlib notebook
#import seaborn as sns

df_ALL = pd.read_sas("./HN16_ALL.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')
df_ffq = pd.read_sas("./hn16_ffq.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')

# 칼럼 저장
# df_ffq.to_csv("df_ffq.csv", encoding='iso-8859-1')
# df_data.to_csv("df_data.csv", encoding='iso-8859-1')

### 데이터 준비
1. 2개의 데이터프레임에서 중복된 칼럼 확인
2. 2번째 데이터프레임에서 키값으로 사용할 ID를 제외한 나머지 중복 칼럼을 삭제
3. ID를 키값으로 2개의 데이터 프레임 머지(left, right, inner, outer, index 등 세팅 확인
4. 정상적으로 데이터프레임 합쳐졌는지, 널값으로 확인
5. 이후 decision tree sklearn regressor를 이용해서 확인(코딩)

In [2]:
# 2개의 리스트에 각각의 데이터프레임의 컬럼명을 저장
list_col_all = df_ALL.columns.values
list_col_ffq = df_ffq.columns.values

# 리스트를 집합으로 변환하여 intersection 사용하여 교집합 구하고 다시 리스트로 반환
list_col_overlab = list(set(list_col_all).intersection(list_col_ffq))
# print(len(list_col_overlab)) # 갯수 확인
# list_col_overlab # 컬럼 확인

In [3]:
# 총 26개의 중복 칼럼명 리스트에서 merge 시 key값으로 사용할 'ID'' 삭제
list_col_overlab.remove('ID')
# print(len(list_col_overlab)) # 갯수 확인
# list_col_overlab # 컬럼 확인

In [4]:
# df_ffq 데이터프레임에서 ID를 제외한 나머지 중복 컬럼 삭제
print(df_ffq.shape)
df_ffq.drop(list_col_overlab, axis=1, inplace=True)
print(df_ffq.shape)

(8150, 434)
(8150, 409)


In [5]:
# 데이터프레임 merge
df_merge = pd.merge(df_ALL, df_ffq, how='inner', on=[
                                  'ID' # 개인 아이디
                                  
                                 ])
print(df_merge.shape)

(8150, 1176)


In [6]:
# merge 데이터프레임 확인
# df_merge.to_csv("df_merge.csv", encoding='iso-8859-1')

In [7]:
# 19세 이상으로만 제한
df_data = df_merge.loc[(18 < df_merge.age), :]

# 혈압치료중인 대상 제외
df_data = df_data.loc[(1 < df_data.DI1_pt) | (df_data.DI1_pt < 1), :]

df_data.shape

(4727, 1176)

In [8]:
# null값 컬럼별 카운트 확인
#df_nan_sort = df_data.isnull().sum()

In [9]:
# null값 많은 컬럼 및 카운트 저장
#df_nan_sort.to_csv("df_nan_sort.csv", encoding='iso-8859-1')

In [10]:
# 모든 값이 null값이 컬럼 삭제
df_data = df_data.dropna(axis=1, how='all')
df_data.shape

(4727, 1122)

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import graphviz
from sklearn import tree
from tqdm import tqdm_notebook

def data_clean(input_list, is_cap=False) :
    tmp_list = []
    for d in input_list :
        try :
            float(d)
            if is_cap :
                tmp_list.append([d])
            else :
                tmp_list.append(d)
        except :
            if is_cap :
                tmp_list.append([-1])
            else :
                tmp_list.append(-1)
    return tmp_list

def saveDTR(input_df, x_name, y_name) :
    arrX1 = data_clean(getattr(input_df,x_name).fillna(-1), is_cap=True)
    arrY = data_clean(getattr(input_df,y_name).fillna(-1))
    
    model = DecisionTreeRegressor(
        criterion = 'mse',
        max_depth=2
    )
    model.fit(arrX1,arrY)

    dot_data = tree.export_graphviz(model,out_file=None,feature_names=[x_name], class_names=[y_name])

    graph = graphviz.Source(dot_data) 
    graph.render("%s+%s" % (x_name,y_name))

    # mse값 계산. 정상적으로 만든건지는 모르겠음..
    modelPrediction = model.predict(arrX1)
    mse = mean_squared_error(arrY,modelPrediction)
    # x_name, y_name, mse값을 mse_list에 append
    mse_list.append([x_name, y_name, mse])
    
skip_list = [
    df_data.HE_dbp.name
    , df_data.HE_sbp.name
]

# saveDTR(df_data, df_data.age.name, df_data.HE_dbp.name)
# saveDTR(df_data, df_data.age.name, df_data.HE_sbp.name)

# mse list 담을 변수 준비
mse_list = []

for x_name in tqdm_notebook(df_data.columns):
    if x_name in skip_list : continue
    saveDTR(df_data, x_name, df_data.HE_dbp.name)
    saveDTR(df_data, x_name, df_data.HE_sbp.name)

# mse_list > dataframe으로 변환해서 csv파일로 저장   
df_mse = pd.DataFrame(mse_list)
df_mse.to_csv("mse_list.csv")